# Importing STACs with PySTAC

The [transaction](https://github.com/radiantearth/stac-api-spec/tree/master/extensions/transaction) extension to the STAC API specification allows adding STAC items to an existing collection (among other things). In this example, we'll add STAC 0.9.0 items to an existing collection in Franklin using [PySTAC](https://github.com/azavea/pystac/).

This example assumes you have Franklin running locally on port 9090. To set up Franklin locally, see the [README](../README.md). If you're trying this example with Franklin running somewhere else, substitute your base url here:

In [1]:
FRANKLIN_HOST = "http://localhost:9090"

## Reading the catalog

The items for this example live in `s3://rasterfoundry-development-data-us-east-1/berlin-catalog/`

```bash
$ aws --profile raster-foundry s3 ls --recursive s3://rasterfoundry-development-data-us-east-1/berlin-catalog/

2020-04-14 11:10:04        461 berlin-catalog/catalog.json
2020-04-14 11:13:41       1097 berlin-catalog/collection.json
2020-04-15 15:23:30  324896818 berlin-catalog/image/berlin-not-cog.tif
2020-04-14 12:59:43       1553 berlin-catalog/image/berlin-sentinel.json
2020-04-15 15:23:30  112040499 berlin-catalog/image/berlin.tif
2020-04-14 10:50:46     587090 berlin-catalog/labels/berlin-labels.tif
2020-04-23 17:17:17       1880 berlin-catalog/labels/bldg-labels.json
2020-04-15 15:24:04    9005460 berlin-catalog/labels/colormap.png
2020-04-15 15:24:06    9005460 berlin-catalog/labels/colormap.tif
2020-04-14 10:50:48     666253 berlin-catalog/predictions/berlin-preds.tif
2020-04-24 15:38:20       2160 berlin-catalog/predictions/bldg-predictions.json
2020-04-24 15:36:54       5367 berlin-catalog/predictions/some-random-boxes.json
```

We'll read the `berlin-sentinel.json` item with a custom `read_text_method` in `STAC_IO` that's aware of how to read from S3.

**Note**: if you don't have a `default` AWS profile, you'll need to set one up using `aws configure` from the [AWS CLI](https://aws.amazon.com/cli/).

In [2]:
import boto3
from pystac import Collection, Item
from pystac.stac_io import STAC_IO
from urllib.parse import urlparse

s3_client = boto3.client('s3')

def read_text_s3(uri: str) -> str:
    parsed = urlparse(uri)
    bucket = parsed.hostname
    key = parsed.path.lstrip("/")
    return s3_client.get_object(Bucket=bucket, Key=key)["Body"].read()

STAC_IO.read_text_method = read_text_s3

item = Item.from_file("s3://rasterfoundry-development-data-us-east-1/berlin-catalog/image/berlin-sentinel.json")
(item, item.get_parent())

(<Item id=image>, <Catalog id=berlin>)

## Find collections in Franklin

Franklin advertises its collections under the `/collections` endpoint. For this example we'll use an existing collection called `berlin` to POST items to.

In [3]:
import requests

[Collection.from_dict(x) for x in requests.get(f"{FRANKLIN_HOST}/collections").json()["collections"]]

[<Collection id=berlin>, <Collection id=berlin building predictions-labels-1>]

## Adding items to Franklin

Franklin accepts POSTs to `/collections/<id>/items` with STAC 0.9.0 items. Since we have one of those, we can add it to the `berlin` collection:

In [4]:
url = f"{FRANKLIN_HOST}/collections/berlin/items"
resp = requests.post(url, json=item.to_dict())
result_item = Item.from_dict(resp.json())
result_item

<Item id=image>

In [5]:
result_item.to_dict()

{'type': 'Feature',
 'stac_version': '0.9.0',
 'id': 'image',
 'properties': {'datetime': '2018-03-03T10:20:19Z'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[13.742925226720015, 52.64467528345354],
    [13.330610521659846, 52.63948385599755],
    [13.340993376571825, 52.363530645448044],
    [13.756192207996433, 52.36860670784946],
    [13.742925226720015, 52.64467528345354]]]},
 'bbox': [13.330610521659846,
  52.363530645448044,
  13.756192207996433,
  52.64467528345354],
 'links': [{'rel': 'parent',
   'href': 'http://localhost:9090/api/collections/berlin',
   'type': 'application/json',
   'label:assets': []},
  {'rel': 'root',
   'href': '../catalog.json',
   'type': 'application/json',
   'label:assets': []},
  {'rel': 'self',
   'href': 's3://rasterfoundry-development-data-us-east-1/berlin-catalog/image/berlin-sentinel.json',
   'type': 'application/json',
   'label:assets': []}],
 'assets': {'sentinel image': {'href': 's3://rasterfoundry-development-data-us-east-1/berl

## More transactions

The transactions endpoint also allows us to update and delete items. Any STAC API that adheres to the transaction extension allows ongoing maintenance of your STAC data as you acquire new data or discover new attributes of your data.

For example, we could update the item we just posted to give it a property `"manuallyCreated": true`:

In [6]:
result_item.properties["manuallyCreated"] = True
result_item.properties

{'datetime': '2018-03-03T10:20:19Z', 'manuallyCreated': True}

In [7]:
item_url = f"{FRANKLIN_HOST}/collections/berlin/items/{result_item.id}"
update_resp = requests.put(item_url, json=result_item.to_dict(), headers={"If-Match": resp.headers["ETag"]})
update_resp.raise_for_status()
get_resp = requests.get(item_url)
get_resp.raise_for_status()
item.from_dict(get_resp.json()).properties

{'datetime': '2018-03-03T10:20:19Z', 'manuallyCreated': True}

Or if we decide that we'd rather just get rid of this item, we can delete it instead:

In [8]:
delete_resp = requests.delete(item_url)
delete_resp.raise_for_status()